In [1]:
import os
import torch
from torch import nn
import numpy as np
from tqdm import tqdm
import torch.nn.functional as F

In [2]:
print(torch.cuda.is_available())

True


In [4]:
def _3d_block(in_size, out_size, kernel_size, stride, padding, bias=False, relu_type='prelu'):
    return nn.Sequential(
        nn.Conv3d(in_size, out_size, kernel_size=kernel_size, stride=stride, padding=padding, bias=bias),
        nn.BatchNorm3d(out_size),
        nn.PReLU(num_parameters=out_size) if relu_type== 'prelu' else nn.ReLU()
    )